In [58]:
import wfdb
import os
import pandas as pd
import wfdb.processing as wp
import numpy as np
import pickle
from biosppy.signals import ecg, tools


import torch
import torch.nn as nn
import torch.utils.data as data
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset
from torch import nn, optim

import pytorch_model_summary

from sklearn.preprocessing import MinMaxScaler as mms

import matplotlib.pyplot as plt
import matplotlib

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIbLE_DEVICES"] = "0"
torch.manual_seed(1234)

In [59]:
pickle_path = "./mit_pickle"
def record_extract(input_path):
    records = open(input_path+"RECORDS","r")
    records_list = []
    for l in records:
        l = l.rstrip()
        records_list.append(l)
    records.close()
    return records_list
#data extract from Physionet

input_path = "../physionet/mit-bih_arr/1.0.0/"
records = open(input_path+"RECORDS","r")
records_list = []
for l in records:
    l = l.rstrip()
    records_list.append(l)
records.close()
print(records_list)
NORMAL_ANN = ['N', 'L', 'R', 'e', 'j']
SUPRA_ANN = ['A', 'a', 'J', 'S']
VENTRI_ANN = ['V', 'E']
FUSION_ANN = ['F']
UNCLASS_ANN = ['/', 'f', 'Q']

['100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '111', '112', '113', '114', '115', '116', '117', '118', '119', '121', '122', '123', '124', '200', '201', '202', '203', '205', '207', '208', '209', '210', '212', '213', '214', '215', '217', '219', '220', '221', '222', '223', '228', '230', '231', '232', '233', '234']


In [60]:
class conv_gru(nn.Module):
    def __init__(self, in_channel, out_channel, batch, hidden_size):
        super(conv_gru, self).__init__()
        
        #input Layer
        self.input_Seq1 = nn.Sequential(
            nn.Conv1d(in_channels=in_channel, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU()
        )
        self.input_Seq2 = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )
        self.input_Seq3 = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )
        self.input_Seq4 = nn.Sequential(
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU()
        )
#         self.lfem_module = LFEM_Stack(6)
        # reshape
        # dropout
        # gru
        # dense 1
        # dense 2
        self.dropout = nn.Dropout(0.3)
        self.gru = nn.GRU(input_size=320,batch_first=True, hidden_size=hidden_size ,num_layers=2, bidirectional=False)
        
        self.dense_Seq = nn.Sequential(
            nn.Linear(256*hidden_size, 256),
            nn.LeakyReLU(inplace=True),
            nn.Linear(256,out_channel)
        )
#         self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        
        x = self.input_Seq1(x)
        x = self.input_Seq2(x)
        x = self.input_Seq3(x)
        x = self.input_Seq4(x)
        x = self.dropout(x)
        x, _ = self.gru(x)
        x = x.reshape(x.size(0),-1)
        x = self.dense_Seq(x)
        
#         return self.softmax(x)
        return x

In [57]:
# Pickle Data Load

def load_data():
    pkl_path = "./pickle/"

    data_x, data_y = [] ,[] 
    for pkl_file in os.listdir(pkl_path):
        with open(pkl_path+pkl_file, "rb") as f:
            data = pickle.load(f)
            for i,d in enumerate(data["Beats"]):
                data_x.append(d)
                data_y.append(data["symbol"][i])
#             print(torch.tensor(data["Beats"]).shape)
#             print(torch.tensor(data["symbol"]).shape)
    return torch.tensor(data_x), torch.tensor(data_y)

x_data, y_data = load_data()
print(x_data.shape, y_data.shape)

train_data = torch.utils.data.TensorDataset(x_data,y_data)
train_len = x_data.shape[0]
# print(train_len)
val_len = int(train_len * 0.2)
# print(val_len)
train_len -= val_len
train_dataset, val_dataset = torch.utils.data.random_split(train_data, [train_len, val_len])
print(train_len, val_len)

lr = 3e-3
epochs = 30
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = conv_gru(in_channel=1,out_channel=6,batch=32,hidden_size=256)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

best_acc, best_epoch = 0, 0
global_step = 0

torch.Size([112599, 320]) torch.Size([112599])
90080 22519


In [61]:
print(len(train_dataset))
print(len(val_dataset))

90080
22519


In [62]:
from tqdm import tqdm
from pytorchtools import EarlyStopping

val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

early_stopping = EarlyStopping(patience = 3, verbose = True)


loss_list = []

for ep in range(epochs):
    train_bar = tqdm(train_loader)
    for step, (x,y) in enumerate(train_bar):
        
        x = x.unsqueeze(dim=1)
#         print("input,",x.shape, y.shape)
        x, y = x.to(device).float(), y.to(device).long()
        model.train()
        logits = model(x)
        loss = criterion(logits, y)
        loss_list.append(loss)
#         print(loss)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
        train_bar.desc = "Train Epoch[{}/{}] loss: {:.3f}".format(ep+1, epochs, loss)
        global_step +=1
    
    model.eval()
    val_loss = 0
    
    val_bar = tqdm(val_loader)
    for step, (x_, y_) in enumerate(val_bar):
        x_ = x_.unsqueeze(dim=1).to(device).float()
        y_pred = model(x_)
        y_ = y_.to(device).long()
        loss = criterion(y_pred, y_)
        
        val_loss += loss.item()
    
    early_stopping(val_loss, model)
    
    if early_stopping.early_stop:
        break

  0%|                                                                                                                                                                     | 0/2815 [00:00<?, ?it/s]

Validation loss decreased (inf --> 1854.467625).  Saving model ...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 704/704 [00:14<00:00, 48.61it/s]


Validation loss decreased (1854.467625 --> 77.519934).  Saving model ...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 704/704 [00:14<00:00, 48.67it/s]


...

EarlyStopping counter: 1 out of 3


Train Epoch[8/30] loss: 0.002:   0%|                                                                                                                              | 2/2815 [00:00<02:43, 17.19it/s]

EarlyStopping counter: 2 out of 3


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 704/704 [00:14<00:00, 48.53it/s]

EarlyStopping counter: 3 out of 3
